In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import os

In [5]:
def get_file_names(path_to_files):
    """
    
    """
    result = []
    for _, path in path_to_files.items():
        try:
            files = os.listdir('data/{}'.format(path))
            tickers = [file.strip('.csv') for file in files]
            result.append(tickers)
        except:
            continue
    return result

In [6]:
def list_intersection(file_names):
    """
    
    """
    max_length = 0
    max_idx = 0
    for idx in range(len(file_names)):
        cur_len = len(file_names[idx])
        if cur_len > max_length:
            max_idx = idx
            max_length = cur_len
    
    s = set(file_names[max_idx])
    return list(s.intersection(*file_names))

In [7]:
def get_feature_names(path_to_files, common_element):
    """
    
    """
    features = []
    for _, path in path_to_files.items():
        cols = pd.read_csv('data/{0}/{1}.csv'.format(path, common_element))
        features.append(cols.columns.values)
    return features

In [8]:
def create_base_df(path_to_files, tickers):
    """
    
    """
    df = pd.read_csv('{0}{1}.csv'.format(path_to_files, tickers[0]))
    df['ticker'] = tickers[0]
    if len(tickers) == 1:
        return df
    
    idx = len(tickers)
    for ticker in tickers[1:]:
        try:
            current_df = pd.read_csv('{0}{1}.csv'.format(path_to_files,
                                                         ticker))
            current_df['ticker'] = ticker
            df = pd.concat([df, current_df], ignore_index=True)
        except:
            print(ticker + ' not found!') 
            
    df.reset_index(drop=True)
    df['Release Date'] = pd.to_datetime(df['Release Date'], format='%b %d, %Y')
    df['Period End'] = pd.to_datetime(df['Period End'])
    return df

In [9]:
def add_fundamentals(df, tickers, path_to_files, groups):
    """

    """
    # add columns we want to use
    new_groups = []
    for group_num in range(len(groups)):
        new_groups.append(groups[group_num][1:-1])
        all_zeros = np.zeros(len(new_groups[-1]))
        match_group_val = dict(zip(new_groups[-1], all_zeros))
        df = df.assign(**match_group_val)

    group_num = 0
    for group_name, path in path_to_files.items():
        features = new_groups[group_num]
        for ticker in tickers:
            indexes = df[df['ticker']==ticker].index.values
            calendar = df[df['ticker']==ticker]['Period End']
            data = pd.read_csv('data/{0}/{1}.csv'.format(path, ticker))
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
            for idx in indexes:
                cond_df = data[(data['date'].dt.year==calendar[idx].year)\
                             & (data['date'].dt.month==calendar[idx].month)]
                if not cond_df.empty:
                    df.loc[idx,features] = cond_df.loc[:,features].values[0]
            print(group_name + ' data for ' + ticker + ' added!')
        print('\n' + group_name + ' features have been added!\n')
        group_num += 1
    return df

In [10]:
def assign_values(df, n_before, k_after, idx = None, 
                  vals = None, total_inds = None):
    """
    
    """   
    if idx is None:
        prev_days = [str(n) + 'd_before' for n in range(n_before, 0, -1)]
        next_days = [str(k) + 'd_after' for k in range(1, k_after + 1)]
        all_days = prev_days + ['0d_release'] + next_days
        
        quote_inds = ['open', 'high', 'low', 'close', 'volume']
        total_inds = []
        for day in all_days:
            for ind in quote_inds:
                total_inds.append(day + '_' + ind)
        
        init_vals = np.zeros(len(total_inds))
        match_ind_val = dict(zip(total_inds, init_vals))
        df = df.assign(**match_ind_val)        
        return (df, total_inds)
    
    df.loc[idx, total_inds] = vals
    return df,_

In [63]:
def find_quotes(ticker, quotes, release_day, n_days_before, k_days_after):
    """
    """
    first_date = quotes['date'][0]
    last_date = quotes['date'][len(quotes) - 1]
    
    idx_release_df = quotes[quotes['date'] == release_day]
    if idx_release_df.empty:
        out_shape = (n_days_before + k_days_after + 1, quotes.shape[1] - 1)
        return np.zeros(out_shape)

    idx_release_day = idx_release_df.index[0]
    start_date = max(idx_release_day - n_days_before, 0)
    end_date = min(idx_release_day + k_days_after, len(quotes) - 1)
    
    quotes_needed = quotes.iloc[start_date:end_date + 1, 1:]
    if len(quotes_needed) != n_days_before + k_days_after + 1:
        dif_start = idx_release_day - n_days_before
        dif_end = idx_release_day + k_days_after - len(quotes) + 1
        if dif_start < 0:
            additional_rows = np.zeros((-dif_start, quotes_needed.shape[1]))
            additional_df = pd.DataFrame(additional_rows, 
                                         columns=quotes_needed.columns.values)
            quotes_needed = pd.concat([additional_df, quotes_needed])
        if dif_end > 0:
            additional_rows = np.zeros((dif_end, quotes_needed.shape[1]))
            additional_df = pd.DataFrame(additional_rows, 
                                         columns=quotes_needed.columns.values)
            quotes_needed = pd.concat([quotes_needed, additional_df])
    return quotes_needed.values

In [64]:
def add_stock_quotes(df, tickers, n_days_before, k_days_after):
    """
    
    """
    df, total_inds = assign_values(df, n_days_before, k_days_after)

    idx = 0
    for ticker in tickers:
        path = 'data/historical_daily_quotes/{}.csv'.format(ticker)
        quotes = pd.read_csv(path)      
        quotes['date'] = pd.to_datetime(quotes['date'], 
                                          format='%Y-%m-%d')
        
        for _, row in df[idx:].iterrows():
            if row['ticker'] != ticker:
                continue    
            release_date = row['Release Date']
            vals = find_quotes(ticker, quotes, release_date, 
                               n_days_before, k_days_after)
            df,_ = assign_values(df, n_days_before, k_days_after, 
                                 idx, vals.flatten(), total_inds)
            idx += 1
        print(ticker + ' stock quotes have been added!')
    return df

In [24]:
path_to_files = {
    'quotes': 'historical_daily_quotes',
    'estimates': 'estimates',
    'balance': 'fundamentals/balance_sheets',
    'cash': 'fundamentals/cash_flow',
    'enterprise': 'fundamentals/enterprise_value',
    'metrics': 'fundamentals/key_metrics',
    'growth': 'fundamentals/financial_statement_growth'
}

In [127]:
data = pd.read_csv('data_without_quotes.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,Release Date,Period End,EPS,Forecast,Revenue,Forecast.1,ticker,Cash and cash equivalents,...,Gross Profit Growth,Inventory Growth,Net Income Growth,Operating Cash Flow growth,Operating Income Growth,R&D Expense Growth,Receivables growth,SG&A Expenses Growth,Weighted Average Shares Diluted Growth,Weighted Average Shares Growth
0,0,0,2020-02-20,2019-12-01,--,0.28,--,968.66M,VTR,0.0,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000
1,1,1,2019-10-25,2019-09-01,0.23,0.32,983.16M,931.93M,VTR,148063000.0,...,0.0262,0.0,-0.1639,0.1531,-0.0177,0.0,0.0,0.0215,0.0183,0.0457
2,2,2,2019-07-26,2019-06-01,0.58,0.32,950.72M,922.96M,VTR,81987000.0,...,-0.0116,0.0,0.2643,-0.0229,-0.0005,0.0,0.0,0.1752,0.0049,0.0061
3,3,3,2019-04-26,2019-03-01,0.35,0.3,942.87M,911.1M,VTR,82514000.0,...,-0.0357,0.0,0.5982,0.0912,-0.0294,0.0,0.0,0.0965,0.0023,0.0015
4,4,4,2019-02-08,2018-12-01,0.17,0.35,923.26M,909.59M,VTR,72277000.0,...,-0.0495,0.0,-0.8414,0.0500,-0.3965,0.0,0.0,0.1015,0.0001,0.0012


In [128]:
files_in_folders = get_file_names(path_to_files)
common_tickers = list_intersection(files_in_folders)

data = data[data['ticker'].isin(common_tickers)]
data.reset_index(inplace=True, drop=True)
tickers = data['ticker'].unique()

In [129]:
final_data = add_stock_quotes(data, tickers, 5, 5)

VTR stock quotes have been added!
WDC stock quotes have been added!
CMCSA stock quotes have been added!
AMGN stock quotes have been added!
C stock quotes have been added!
RJF stock quotes have been added!
SPG stock quotes have been added!
RHI stock quotes have been added!
L stock quotes have been added!
FIS stock quotes have been added!
ARE stock quotes have been added!
DUK stock quotes have been added!
QRVO stock quotes have been added!
MNST stock quotes have been added!
HSIC stock quotes have been added!
KIM stock quotes have been added!
MTB stock quotes have been added!
KR stock quotes have been added!
PHM stock quotes have been added!
CL stock quotes have been added!
TAP stock quotes have been added!
MTD stock quotes have been added!
AEE stock quotes have been added!
LB stock quotes have been added!
SWK stock quotes have been added!
CVX stock quotes have been added!
KO stock quotes have been added!
GPS stock quotes have been added!
MU stock quotes have been added!
BKNG stock quotes

ZION stock quotes have been added!
LYB stock quotes have been added!
VNO stock quotes have been added!
AAP stock quotes have been added!
CMA stock quotes have been added!
UA stock quotes have been added!
AES stock quotes have been added!
TRV stock quotes have been added!
CDNS stock quotes have been added!
FITB stock quotes have been added!
EXPD stock quotes have been added!
CF stock quotes have been added!
SLB stock quotes have been added!
MLM stock quotes have been added!
DHR stock quotes have been added!
NFLX stock quotes have been added!
EMR stock quotes have been added!
AXP stock quotes have been added!
EXPE stock quotes have been added!
VFC stock quotes have been added!
FDX stock quotes have been added!
GS stock quotes have been added!
OKE stock quotes have been added!
SBUX stock quotes have been added!
APH stock quotes have been added!
TEL stock quotes have been added!
WAT stock quotes have been added!
INTC stock quotes have been added!
AVY stock quotes have been added!
ROST stoc

In [130]:
final_data

,Unnamed: 0,Unnamed: 0.1,Release Date,Period End,EPS,Forecast,Revenue,Forecast.1,ticker,Cash and cash equivalents,...,4d_after_open,4d_after_high,4d_after_low,4d_after_close,4d_after_volume,5d_after_open,5d_after_high,5d_after_low,5d_after_close,5d_after_volume
0,0,0,2020-02-20,2019-12-01,--,0.28,--,968.66M,VTR,0.000000e+00,...,0.00,0.0000,0.000,0.00,0.0,0.00,0.0000,0.0000,0.00,0.0
1,1,1,2019-10-25,2019-09-01,0.23,0.32,983.16M,931.93M,VTR,1.480630e+08,...,65.21,65.7100,64.630,65.10,2122092.0,65.12,65.3000,63.1488,64.03,2114593.0
2,2,2,2019-07-26,2019-06-01,0.58,0.32,950.72M,922.96M,VTR,8.198700e+07,...,67.33,67.9500,66.510,67.67,2069256.0,67.76,69.1800,67.7600,68.83,1749714.0
3,3,3,2019-04-26,2019-03-01,0.35,0.3,942.87M,911.1M,VTR,8.251400e+07,...,61.23,61.9900,60.960,61.53,2198358.0,61.75,62.5800,61.4650,62.48,1767991.0
4,4,4,2019-02-08,2018-12-01,0.17,0.35,923.26M,909.59M,VTR,7.227700e+07,...,64.29,64.9236,64.020,64.74,2043458.0,64.98,65.2500,64.6400,65.21,1925856.0
5,5,5,2018-10-26,2018-09-01,0.28,0.36,936.54M,908.79M,VTR,8.610700e+07,...,58.17,58.6100,57.510,58.11,2610532.0,57.79,57.7900,55.6400,56.59,3015300.0
6,6,6,2018-07-27,2018-06-01,0.46,0.37,942.3M,906.16M,VTR,9.368400e+07,...,56.68,57.1900,56.420,56.49,1454377.0,56.68,58.3400,56.6250,58.00,2379265.0
7,7,7,2018-04-27,2018-03-01,0.22,0.42,943.71M,861.73M,VTR,9.254300e+07,...,52.12,52.6000,51.820,52.53,2773771.0,52.45,53.2300,52.3800,53.15,2357698.0
8,8,8,2018-02-09,2017-12-01,1.09,0.71,895.3M,875.65M,VTR,8.135500e+07,...,49.62,50.7500,49.620,50.64,2632840.0,50.79,51.6400,50.7100,51.39,2448403.0
9,9,9,2017-10-27,2017-09-01,0.44,0.45,899.9M,882.25M,VTR,8.510000e+07,...,63.24,64.1800,63.170,63.84,1689402.0,63.50,64.5500,63.4850,64.10,1243378.0


In [132]:
final_data.to_csv('data_final_5d.csv', index=False)